In [1]:
import tensorflow as tf
from numpy.random import RandomState

In [2]:
# 实战Google深度学习实战
# 回归问题损失函数的定义,使用单层神经网络 P80
batch_size = 8

In [3]:
# 2个输入节点
x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
# 回归问题一般只有1个输出节点
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

# 定义一个单层的神经网络前向传播过程, 这里就是简单加权和
# 满足正态分布,均方差为1的数据
w1 = tf.Variable(tf.random_normal([2,1], stddev=1, seed=1))
y = tf.matmul(x, w1)

# 定义损失, 预测多了和预测少了的成本, 根据业务实际情况选择参数, 这里预测多了损失为1, 预测少了损失为10
loss_less = 10
loss_more = 1
loss = tf.reduce_sum(tf.where(tf.greater(y,y_), (y-y_)*loss_more, (y_-y)*loss_less))

"""
使用L2正则化的损失函数
由2部分构成, 第一部分为均方差损失函数, 第二个部分就是正则化,它防止模型过度模拟训练数据中的随机噪音. 
lambda参数表示了正则化项的权重, w为需要计算正则化损失的参数.
loss = tf.reduce_mean(tf.square(y_ - y) + tf.contrib.layers.l2_regularizer(lambda)(w))

"""

train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

"""
# 使用指数衰减的学习率
global_step = tf.Variable(0)
# 通过exponential_decay函数生成学习率, 设置了阶梯下降,每训练100轮后学习率乘以0.96
learning_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.96, staircase=True)
# 使用指数衰减的学习率, 在minimize函数中传入global_step将自动更新global_step参数,从而使得学习率也得到相应更新
learning_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
"""




'\n# \xe4\xbd\xbf\xe7\x94\xa8\xe6\x8c\x87\xe6\x95\xb0\xe8\xa1\xb0\xe5\x87\x8f\xe7\x9a\x84\xe5\xad\xa6\xe4\xb9\xa0\xe7\x8e\x87\nglobal_step = tf.Variable(0)\n# \xe9\x80\x9a\xe8\xbf\x87exponential_decay\xe5\x87\xbd\xe6\x95\xb0\xe7\x94\x9f\xe6\x88\x90\xe5\xad\xa6\xe4\xb9\xa0\xe7\x8e\x87, \xe8\xae\xbe\xe7\xbd\xae\xe4\xba\x86\xe9\x98\xb6\xe6\xa2\xaf\xe4\xb8\x8b\xe9\x99\x8d,\xe6\xaf\x8f\xe8\xae\xad\xe7\xbb\x83100\xe8\xbd\xae\xe5\x90\x8e\xe5\xad\xa6\xe4\xb9\xa0\xe7\x8e\x87\xe4\xb9\x98\xe4\xbb\xa50.96\nlearning_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.96, staircase=True)\n# \xe4\xbd\xbf\xe7\x94\xa8\xe6\x8c\x87\xe6\x95\xb0\xe8\xa1\xb0\xe5\x87\x8f\xe7\x9a\x84\xe5\xad\xa6\xe4\xb9\xa0\xe7\x8e\x87, \xe5\x9c\xa8minimize\xe5\x87\xbd\xe6\x95\xb0\xe4\xb8\xad\xe4\xbc\xa0\xe5\x85\xa5global_step\xe5\xb0\x86\xe8\x87\xaa\xe5\x8a\xa8\xe6\x9b\xb4\xe6\x96\xb0global_step\xe5\x8f\x82\xe6\x95\xb0,\xe4\xbb\x8e\xe8\x80\x8c\xe4\xbd\xbf\xe5\xbe\x97\xe5\xad\xa6\xe4\xb9\xa0\xe7\x8e\x87\xe4\xb9\x9f\xe5

In [4]:
# 通过随机数生成一个模拟数据集, 128 x 2
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
X.shape
# X

(128, 2)

In [5]:
# 设置回归的正确值为两个输入的和加上一个随机量, 是为了加入不可预测的噪音, 否则不同损失函数的意义就不大了.因为不同损失函数都会
# 在能完全预测正确时最低.一般噪音为一个均值为0的小量, 所以这里设置为-0.05 ~ 0.05的随机数
Y = [[x1 + x2 + rdm.rand()/10.0-0.05] for (x1,x2) in X]
# Y


In [6]:
# 训练神经网络
# weights= tf.constant([[l.O, -2 . 0), (-3 . 0, 4 . 0)))

with tf.Session() as sess:
    """
    L1和L2损失函数
    # 输出为（|1|+|-2|＋|-3|+|4|)x0.5=5 。其中 0.5 为正则化项的权重。 
    # print sess.run(tf.contrib.layers.l1_regularizer(.5)(weights)) 
    # 输出为（ 1^2+ (-2)^2+ (-3)^2+4^2 )/2 × 0.5=7.5
    # print sess.run(tf.contrib.layers.l2_regularizer(.5)(weights))
    """
    
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 5000
    for i in range(STEPS):
        start = (i*batch_size) % dataset_size
        end = min(start+batch_size, dataset_size)
        sess.run(train_step,
                feed_dict={x:X[start:end], y_:Y[start:end]})
        if i % 500==0:
            print sess.run(w1)

[[-0.81031823]
 [ 1.4855988 ]]
[[-0.34678397]
 [ 1.9075472 ]]
[[0.01247112]
 [2.1385448 ]]
[[0.27079922]
 [2.212271  ]]
[[0.45567414]
 [2.1706066 ]]
[[0.59347594]
 [2.0449932 ]]
[[0.69968724]
 [1.8465308 ]]
[[0.8010899]
 [1.5939   ]]
[[0.89886665]
 [1.2973602 ]]
[[1.0170875]
 [1.0466708]]


In [7]:
# 运行结果[[1.0194756] [1.043531 ]] 也就是预测函数为1.02x1+1.04x2 , 比x1+x2大, 因为损失函数中指定预测少了的损失更大
# 如果使用均方差作为损失函数, 那么w1会是[0.9743, 1.0243], 使用这个损失函数会尽量让预测值离标准答案更近. 
# 对于相同的神经网络, 不同的损失函数会对训练得到的模型产生重要影响



In [15]:
# 通过集合计算一个5层神经网络带L2正则化的损失函数, 定义网络结构和计算损失函数分开  

# 获取－层神经网络边上的权重 并将这个权重的 L2 正则化损失加入名称为 ’ losses ’的集合中

def get_weight (shape, lambdas):
    # 生成一个变量。
    var = tf.Variable(tf.random_normal(shape), dtype = tf.float32)
    # add to collection 函数将这个新生成变量的 L2 正则化损失项加入集合。 
    # 这个函数的第一个参数 ’ losses ’ 是集合的名字， 第二个参数是要加入这个集合的内容。 
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambdas)(var))
    # 返回生成的变量。
    return var

x = tf.placeholder(tf.float32, shape=(None,2))
y_ = tf.placeholder(tf.float32, shape=(None,1))
batch_size = 8
# 定义了每一层网络中节点的个数 。 
layer_dimension = [2 , 10 , 10 , 10 , 1] 
# 神经网络的层数。 
n_layers = len(layer_dimension)

# 这个变量维护前向传播时最深层的节点， 开始的时候就是输入层。

cur_layer = x 

# 当前层的节点个数。 
in_dimension = layer_dimension[0]

# 通过一个循环来生成5层全连接的冲经网络结构。

for i in range(1, n_layers):
    # layer dimension[i ）为下一层的节点个数。 
    out_dimension = layer_dimension[i] 
    #生成当前层中权重的变量， 并将这个变量的L2正则化损失加入计算图上的集合。 
    weight= get_weight([in_dimension, out_dimension], 0.001)
    bias = tf.Variable(tf.constant(0.1, shape=[out_dimension]))

    # 使用 ReLU 激活函数。
    cur_layer= tf.nn.relu(tf.matmul(cur_layer, weight) + bias)
    #进入下一层之前将下一层的节点 个数更新为当前层节点个数。 
    in_dimension = layer_dimension[i]

# 在定义神经网络前向传播的同时 已经将所有的 L2 正则化损失加入了图上的集合
# 这里只需要计算刻画模型在训练 数据上表现的损失函数。

mse_loss= tf.reduce_mean(tf.square(y_ - cur_layer))

#将均方误差损失函数加入损失集合。
tf.add_to_collection ('losses', mse_loss)

# get_collection 返回一个列表 ，这个列表是所有这个集合中的元素 。
# 在这个样例中, 这些元素就是损失函数的不同部 分， 将它们加起来就可以得到最终的损 失函数。

loss = tf.add_n(tf.get_collection('losses'))
print loss

Tensor("AddN_3:0", shape=(), dtype=float32)


In [19]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    init_ops = tf.Variable(tf.constant(0.1, shape=[2]))
    print sess.run(init_ops)
    

FailedPreconditionError: Attempting to use uninitialized value Variable_37
	 [[Node: _retval_Variable_37_0_0 = _Retval[T=DT_FLOAT, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_37)]]